# Notebook to drive Vegetation Edge extraction from Satellite Images
The programming language we are using is called Python. The code has all been written and this notebook will guide you through modifying the analysis for your own area of interest, and executing the analysis.

**To run a code block, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `In [1]:` when the code is finished. *The order in which you execute the code blocks matters, they must be run in sequence.* Some cells are optional however! Please read the descriptions to decide if you need to run each cell.

If the heading above a cell is <font color='red'>IN RED</font>, this means you should edit this cell with your own requirements and settings.

Inside blocks of python code there are comments indicated by lines that start with `#`. These lines are not computer code but rather comments providing information about what the code is doing to help you follow along and troubleshoot. 


### Import Python packages/modules
Load in the packages we'll need for running the tool.

In [ ]:
import os
import glob
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
plt.ion()
from datetime import datetime
from Toolshed import Download, Toolbox, VegetationLine, Plotting, PlottingSeaborn, Transects
import ee
import geopandas as gpd
import geemap

ee.Initialize()

### <font color='red'>User requirements</font>
Edit these variables to set up your site of interest, including desired coordinate projections, site name and bounding box, satellite platforms (`L5` is Landsat 5, `L8` is Landsat 8, and `S2` is Sentinel 2) and date range of interest.

In [ ]:
# Define AOI using coordinates of a rectangle
# The points represent the corners of a bounding box that go around your site
sitename = 'SITENAME'

# Date range
dates = ['2021-05-01', '2021-07-02']

# Satellite missions
# Input a list of containing any/all of 'L5', 'L7', 'L8', 'L9', 'S2', 'PSScene4Band'
# L5: 1984-2013; L7: 1999-2017 (SLC error from 2003); L8: 2013-present; S2: 2014-present; L9: 2021-present
sat_list = ['L5','L8','S2']

# Cloud threshold for screening out cloudy imagery (0.5 or 50% recommended)
cloud_thresh = 0.5

# Extract shoreline (wet-dry boundary) as well as veg edge
wetdry = True

# Reference shoreline/veg line shapefile name (should be stored in a folder called referenceLines in Data)
# Line should be ONE CONTINUOUS linestring along the shore, stored as a shapefile in WGS84 coord system
referenceLineShp = 'SITENAME_refLine.shp'
# Maximum amount in metres by which to buffer the reference line for capturing veg edges within
max_dist_ref = 150

### Set Up Site Directory
Make a series of directories for the new sitename.

In [ ]:
# Directory where the data will be stored
filepath = Toolbox.CreateFileStructure(sitename, sat_list)

### Reference Shore Option 1: Define AOI from the bounding box of the <font color='red'>reference shoreline shapefile</font>
You shouldn't need to change anything here; the `referenceLineShp` comes from the <font color='red'>User requirements</font> cell

In [ ]:
# Return AOI from reference line bounding box and save AOI folium map HTML in sitename directory
referenceLinePath = os.path.join(filepath, 'referenceLines', referenceLineShp)
referenceLineDF = gpd.read_file(referenceLinePath)
polygon, point, lonmin, lonmax, latmin, latmax = Toolbox.AOIfromLine(referenceLinePath, max_dist_ref, sitename)

# It's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = Toolbox.smallest_rectangle(polygon)

### Reference Shore Option 2: <font color='red'>Draw desired AOI on map</font>
Use the polygon or rectangle tool in the Leaflet map window (that will appear below the next cell) to draw out your area of interest. Defining only one feature is valid at present. Once you are happy with your AOI, run the next cell to save the feature as your AOI for gathering satellite metadata within.

In [ ]:
# Run this cell to generate an interactive map
Map = geemap.Map(center=[0,0],zoom=2)
Map.add_basemap('HYBRID')
Map

In [ ]:
# Run this cell after defining your region of interest
roi = Map.user_roi.geometries().getInfo()[0]['coordinates']
polygon = [[roi[0][0],roi[0][3],roi[0][1],roi[0][2]]]
point = ee.Geometry.Point(roi[0][0])

polygon, point, BBoxGDF = Toolbox.AOIfromLeaflet(polygon, point, sitename)
NewBBox = BBoxGDF.to_json()


In [ ]:
NewBBox

### Reference Shore Option 3: Define AOI from <font color='red'>user defined coordinates</font> 
Make sure you get the latitude and longitude around the right way, and minimum and maximum correct (including any - signs)

In [ ]:
# EDIT THESE to define an AOI bounding box (minimum and maxiumum latitudes and longitudes)
lonmin, lonmax = -2.84869, -2.79878
latmin, latmax = 56.32641, 56.39814

# Return AOI after checking coords and saving folium map HTML in sitename directory
polygon, point = Toolbox.AOI(lonmin, lonmax, latmin, latmax, sitename)

# It's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = Toolbox.smallest_rectangle(polygon)

### Compile Input Settings for Gathering Imagery

In [ ]:
if len(dates)>2:
    daterange='no'
else:
    daterange='yes'
years = list(Toolbox.daterange(datetime.strptime(dates[0],'%Y-%m-%d'), datetime.strptime(dates[-1],'%Y-%m-%d')))

# Put all the inputs into a dictionary
inputs = {'polygon': polygon, 'dates': dates, 'daterange':daterange, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

### Image Retrieval
Before downloading the images, check how many images are available for your inputs

In [ ]:
inputs = Download.check_images_available(inputs)

### Image Download
Compile the metadata from the Google Earth Engine server

In [ ]:
Sat = Download.RetrieveImages(inputs)
metadata = Download.CollectMetadata(inputs, Sat)

### Local Image Retrieval (Planet)
For the time being, Planet API is very slow at downloading files (depending on the size of your area). You may want to use the [Planet online data](https://developers.planet.com/docs/apps/explorer/) download portal and move the files to local folders that will have been created at the Toolbox.CreateFileStructure() step. 
- Move the image TIFFs into 'Data/YOURSITENAME/local_images/PlanetScope';
- and the respective cloud masks into 'Data/YOURSITENAME/local_images/PlanetScope/cloudmasks'.
- You can move any leftover extra files into 'Data/YOURSITENAME/AuxiliaryImages'

In [ ]:
# If you want to include Landsat 7 but DON'T want to include Scan Line Corrector affected images, set SLC=False
Sat = Download.RetrieveImages(inputs, SLC=False)
metadata = Download.CollectMetadata(inputs, Sat)

### <font color='red'>Vegetation Edge Settings</font>
ONLY EDIT THESE IF ADJUSTMENTS ARE NEEDED; for example, if you are getting no/disjointed edges, or  if you want to adjust the threshold used for detection in each image.

In [ ]:
BasePath = 'Data/' + sitename + '/lines'

if os.path.isdir(BasePath) is False:
    os.mkdir(BasePath)

projection_epsg, _ = Toolbox.FindUTM(polygon[0][0][1],polygon[0][0][0])

settings = {
    # general parameters:
    'cloud_thresh': cloud_thresh,        # threshold on maximum cloud cover
    'output_epsg': projection_epsg,     # epsg code of spatial reference system desired for the output   
    'wetdry': wetdry,              # extract wet-dry boundary as well as veg
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 200,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 250,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 500,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    # add the inputs defined previously
    'inputs': inputs,
    'projection_epsg': projection_epsg,
    'year_list': years
}


### <font color='red'>Tidal Information</font>
Compute tides from FES2014 or FES2022 data which is downloaded from the pyFES server. Only relevant for shoreline processing (as it is used to correct for the effects of tides on the cross-shore waterline position). 

(ONLY RUN IF YOU HAVE `pyfes` INSTALLED AND WANT TIDAL INFO SAVED. If you do, change `tidepath` to the path to your `aviso-fes` folder, see the README for details)

Note: FES2022 is more accurate than FES2014 but takes several minutes longer to compute.

In [ ]:
if wetdry is True:
    tidepath = "../aviso-fes/data/fes2014"
    tideoutpath = os.path.join(settings['inputs']['filepath'],'tides',
                            settings['inputs']['sitename']+'_tides_'+
                            settings['inputs']['dates'][0]+'_'+settings['inputs']['dates'][1]+'.csv')
    daterange = dates
    tidelatlon = [(latmin+latmax)/2, (lonmin+lonmax)/2] # centre of bounding box
    Toolbox.ComputeTides(settings,tidepath,tideoutpath,daterange,tidelatlon) 

### Vegetation Edge Reference Line Load-In
Read in a shapefile representing the rough edge of vegetation that you want to investigate along. Does not neet to be accurate as it is used to create a coastal buffer for constraining the edge extraction.

In [ ]:
referenceLinePath = os.path.join(inputs['filepath'], 'referenceLines', referenceLineShp)
referenceLine, ref_epsg = Toolbox.ProcessRefline(referenceLinePath,settings)

settings['reference_shoreline'] = referenceLine
settings['ref_epsg'] = ref_epsg
# Distance to buffer reference line by (this is in metres)
settings['max_dist_ref'] = max_dist_ref

### <font color='red'>Reference Image for Coregistration</font>
You can now coregister your satellite images using AROSICS. If you want to try coregistering your images to improve timeseries accuracy, <font color='red'>provide a filepath to a reference RGB image.</font>

In [ ]:
settings['reference_coreg_im'] = None # leave as None if no coregistration is to be performed

### Vegetation Line Extraction
**OPTION 1**: Run the extraction tool and return output veg edges as a dictionary of lines with associated image info attached.


In [ ]:
output, output_latlon, output_proj = VegetationLine.extract_veglines(metadata, settings, polygon, dates)

### Vegetation Line Extraction Load-In
**OPTION 2**: Load in pre-existing output file generated from a previous run of the vegetation edge extraction tool.


In [ ]:
output, output_latlon, output_proj = Toolbox.ReadOutput(inputs)

# Remove Duplicate Lines
# For images taken on the same date by the same satellite, keep only the longest line
output = Toolbox.RemoveDuplicates(output) 

### Save Veglines as Local Shapefiles
You **DO NOT** need to run this again if you have already exported a previous run to a shapefile, especially not if you have then edited/tidied any unwanted lines for further analysis in an external GIS software (as rerunning this cell will overwrite your edits!).

In [ ]:
# Save output veglines 
Toolbox.SaveConvShapefiles(output, BasePath, sitename, settings['output_epsg'])
# Save output shorelines if they were generated
if settings['wetdry'] == True:
    Toolbox.SaveConvShapefiles_Water(output, BasePath, sitename, settings['output_epsg'])

### <font color='red'>Define Settings for Cross-shore Transects</font>
Edit me to define the characteristics of the cross-shore transects generated for intersecting with the extracted vegetation edges.

In [ ]:
SmoothingWindowSize = 21 
NoSmooths = 100
TransectSpacing = 10
DistanceInland = 100
DistanceOffshore = 100

# Provide average beach slope (tanBeta) for site, for calculating corrected beach widths
# Set to 'None' if you want to use CoastSat.slope to calculate per-transect slopes for correcting with
beachslope = None

### Create Cross-shore Transects
Generate transects based on the above settings.

In [ ]:
VegBasePath = 'Data/' + sitename + '/lines'
VeglineShp = glob.glob(BasePath+'/*veglines.shp')
VeglineGDF = gpd.read_file(VeglineShp[0])
VeglineGDF = VeglineGDF.sort_values(by='dates') # sort GDF by dates to ensure transect intersects occur in chronological order
VeglineGDF = VeglineGDF.reset_index(drop=True) # reset GDF index after date sorting
if settings['wetdry'] == True:
    WaterlineShp = glob.glob(BasePath+'/*waterlines.shp')
    WaterlineGDF = gpd.read_file(WaterlineShp[0])
    WaterlineGDF = WaterlineGDF.sort_values(by='dates') # as above with VeglineGDF date sorting
    WaterlineGDF = WaterlineGDF.reset_index(drop=True)
# Produces Transects for the reference line
TransectSpec =  os.path.join(BasePath, sitename+'_Transects.shp')

# If transects already exist, load them in
if os.path.isfile(TransectSpec[:-3]+'pkl') is False:
    TransectGDF = Transects.ProduceTransects(settings, SmoothingWindowSize, NoSmooths, TransectSpacing, DistanceInland, DistanceOffshore, VegBasePath, referenceLineShp)
else:
    print('Transects already exist and were loaded')
    with open(TransectSpec[:-3]+'pkl', 'rb') as Tfile: 
        TransectGDF = pickle.load(Tfile)
    
# make new transect intersections folder
if os.path.isdir(os.path.join(filepath, sitename, 'intersections')) is False:
    os.mkdir(os.path.join(filepath, sitename, 'intersections'))

### Transect-Veg Intersections
Create (or load existing) a GeoDataFrame holding intersections with veg edges along each transect.

In [ ]:
if os.path.isfile(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_intersects.pkl')):
    print('Transect Intersect GDF exists and was loaded')
    with open(os.path.join
              (filepath , sitename, 'intersections', sitename + '_transect_intersects.pkl'), 'rb') as f:
        TransectInterGDF = pickle.load(f)
else:
    # Get intersections
    TransectInterGDF = Transects.GetIntersections(BasePath, TransectGDF, VeglineGDF)
    # Save newly intersected transects as shapefile
    TransectInterGDF = Transects.SaveIntersections(TransectInterGDF, VeglineGDF, BasePath, sitename)
    # Repopulate dict with intersection distances along transects normalised to transect midpoints
    TransectInterGDF = Transects.CalculateChanges(TransectInterGDF)
    
    with open(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_intersects.pkl'), 'wb') as f:
        pickle.dump(TransectInterGDF, f)

### Transect-Water Intersections
Create (or load existing) a GeoDataFrame holding intersections with waterlines along each transect. 

In [ ]:
if os.path.isfile(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_water_intersects.pkl')):
    print('Transect Intersect + Water GDF exists and was loaded')
    with open(os.path.join
              (filepath , sitename, 'intersections', sitename + '_transect_water_intersects.pkl'), 'rb') as f:
        TransectInterGDFWater = pickle.load(f)
else:        
    if settings['wetdry'] == True:
        TransectInterGDFWater = Transects.GetWaterIntersections(BasePath, TransectGDF, TransectInterGDF, WaterlineGDF, settings, output)  
    
    with open(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_water_intersects.pkl'), 'wb') as f:
        pickle.dump(TransectInterGDFWater, f)


### Transect-Waves Intersections
Create (or load existing) a GeoDataFrame holding intersections with topographic data along each transect. This is for comparing veg edge positions with nearshore wave conditions at the time the image was taken. NOTE: this requires you to have a Copernicus Marine Service (CMEMS) account with access to their hindcast model, as you will be asked for a username and password. This should also be run before the tidal corrections and beach with steps if you want to include runup in your corrections.

In [ ]:

if os.path.isfile(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_wave_intersects.pkl')):
    print('Transect Intersect + Wave GDF exists and was loaded')
    with open(os.path.join
              (filepath , sitename, 'intersections', sitename + '_transect_wave_intersects.pkl'), 'rb') as f:
        TransectInterGDFWave = pickle.load(f)
else:
    TransectInterGDFWave = Transects.WavesIntersect(settings, TransectInterGDF, BasePath, output, lonmin, lonmax, latmin, latmax)
    
    with open(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_wave_intersects.pkl'), 'wb') as f:
        pickle.dump(TransectInterGDFWave, f)

### Additional wave-based WL metrics
This is for comparing shoreline change with vegetation change, and for quantifying the beach width between the two for each image. If you would like to include runup in your waterline corrections, add `TransectInterGDFWave` to `GetWaterIntersections()`:

```TransectInterGDFWater = Transects.GetWaterIntersections(BasePath, TransectGDF, TransectInterGDF, WaterlineGDF, settings, output, TransectInterGDFWave, beachslope)```

If you want to include runup AND calculate slopes using CoastSat.slope (recommended), exclude the `beachslope` variable:

`TransectInterGDFWater = Transects.GetWaterIntersections(BasePath, TransectGDF, TransectInterGDF, WaterlineGDF, settings, output, TransectInterGDFWave)`

In [ ]:

if 'wlcorrdist' not in TransectInterGDFWater.columns:
    # Tidal correction to get corrected distances along transects
    TransectInterGDFWater = Transects.WLCorrections(settings, output, TransectInterGDFWater, TransectInterGDFWave)     
    # Calculate width between VE and corrected WL
    TransectInterGDFWater = Transects.CalcBeachWidth(settings, TransectGDF, TransectInterGDFWater)
    # Calculate rates of change on corrected WL and save as Transects shapefile
    TransectInterGDFWater = Transects.SaveWaterIntersections(TransectInterGDFWater, WaterlineGDF,  BasePath, sitename)
    with open(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_water_intersects.pkl'), 'wb') as f:
        pickle.dump(TransectInterGDFWater, f)

### Transect-Topo Intersections
Create (or load existing) a GeoDataFrame holding intersections with topographic data along each transect. This is for comparing veg edge positions with dune slopes. <font color='red'>Edit the TIF filename</font> if you want to run this intersection process.

In [ ]:
# EDIT ME: Path to slope raster for extracting slope values
TIF = '/path/to/Slope_Raster.tif'

if os.path.isfile(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_topo_intersects.pkl')):
    print('Transect Intersect + Topo GDF exists and was loaded')
    with open(os.path.join
              (filepath , sitename, 'intersections', sitename + '_transect_topo_intersects.pkl'), 'rb') as f:
        TransectInterGDFTopo = pickle.load(f)
else:
    # Update Transects with Transition Zone widths and slope if available
    TransectInterGDFTopo = Transects.TZIntersect(settings, TransectInterGDF, VeglineGDF, BasePath)
    TransectInterGDFTopo = Transects.SlopeIntersect(settings, TransectInterGDFTopo, VeglineGDF, BasePath, TIF)
    
    with open(os.path.join(filepath, sitename, 'intersections', sitename + '_transect_topo_intersects.pkl'), 'wb') as f:
        pickle.dump(TransectInterGDFTopo, f)

### <font color='red'>Plotting</font>
Edit the desired Transect IDs to plot timeseries of veg change and beach width change across 

In [ ]:
# Timeseries Plotting

# EDIT ME: Select transect ID to plot
# You can plot subplots within a list of plot IDs, e.g. [[sub1, sub2], plot2]
# You can also comment Line 1 out and uncomment Line 2 to create plots for ALL Transect IDs
# NOTE: If you want to plot ALL transects, it's recommended you switch ShowPlot=False

TransectIDs = [[25,30,35],50,75] # Line 1
# TransectIDs = list(TransectInterGDF['TransectID']) # Line 2

for TransectID in TransectIDs:
    # Plot timeseries of cross-shore veg position
    Plotting.VegTimeseries(sitename, TransectInterGDF, TransectID, Hemisphere='N', ShowPlot=True)
    # If plotting veg and water lines together
    if settings['wetdry']:
        Plotting.VegWaterTimeseries(sitename, TransectInterGDFWater, TransectID, Hemisphere='N', ShowPlot=True)


In [ ]:
# Beach Width Plotting

# Select transect ID to plot
TransectIDs = [[25,30,35],50,75]
for TransectID in TransectIDs:
    # Plot timeseries of cross-shore width between water edge and veg edge 
    Plotting.WidthTimeseries(sitename, TransectInterGDFWater, TransectID, Hemisphere='N')


## OPTIONAL: Validation
### <font color='red'>Validation Settings</font>
Most likely you won't need to validate your lines, but if you do, edit these parameters.

In [ ]:
# Name of date column in validation edges shapefile (case sensitive!) 
DatesCol = 'Date'
ValidationShp = './Validation/StAndrews_Veg_Edge_combined_2007_2022_singlepart.shp'
# EDIT ME: List of transect ID tuples (startID, finishID)
TransectIDList = [(595,711),(726,889),(972,1140),(1141,1297)]

In [ ]:
# Satellite Edges Validation
validpath = os.path.join(os.getcwd(), 'Data', sitename, 'validation')

if os.path.isfile(os.path.join(validpath, sitename + '_valid_intersects.pkl')):
    print('ValidDict exists and was loaded')
    with open(os.path.join(validpath, sitename + '_valid_intersects.pkl'), 'rb') as f:
        ValidInterGDF = pickle.load(f)
else:
    ValidInterGDF = Transects.ValidateSatIntersects(sitename, ValidationShp, DatesCol, TransectGDF, TransectInterGDF)
    with open(os.path.join(validpath, sitename + '_valid_intersects.pkl'), 'wb') as f:
        pickle.dump(ValidInterGDF, f)


# Quantify errors between validation and satellite derived lines
for TransectIDs in TransectIDList:
    Toolbox.QuantifyErrors(sitename, VeglineGDF,'dates',ValidInterGDF,TransectIDs)

### <font color='red'>Validation Plots</font>
Plot violin plots, probability density functions and regression lines of the cross-shore distance along each transect between each satellite-derived veg edge and validation veg edge.

In [ ]:
# EDIT ME: List of transect ID tuples (startID, finishID)
TransectIDList = [(0,1741)]

for TransectIDs in TransectIDList:
    PlotTitle = 'Accuracy of Transects ' + str(TransectIDs[0]) + ' to ' + str(TransectIDs[1])
    PlottingSeaborn.SatViolin(sitename,VeglineGDF,'dates',ValidInterGDF,TransectIDs, PlotTitle)
    PlottingSeaborn.SatPDF(sitename,VeglineGDF,'dates',ValidInterGDF,TransectIDs, PlotTitle)
    Plotting.SatRegress(sitename,VeglineGDF,'dates',ValidInterGDF,TransectIDs, PlotTitle)